In [2]:
import numpy as np
import scipy.signal as sig

In [3]:
def coeffs_modaux_bois(c0, l, rc, damping=False,ray=False):
    
    f = np.linspace(20, 10000, 5000)
    om = 2*np.pi*f
    k = om/c0
    pi = np.pi
    k = 2 * pi * f / c0
    #Z_c = rho_air * c0 / S_cylinder
    Z_c = 1  # car adimensionné ? 
    
    if damping:
        gamma = 1j * k + (1 + 1j) * 3e-5 * np.sqrt(f) / rc
        A = np.cosh(gamma * l)
        B = Z_c * np.sinh(gamma * l)
        C = 1 / Z_c * np.sinh(gamma * l)
        D = np.cosh(gamma * l)
    else:
        A = np.cos(k * l)
        B = 1j * Z_c * np.sin(k * l)
        C = 1j / Z_c * np.sin(k * l)
        D = np.cos(k * l)
        
    if ray :
        Z_L = np.square(k*rc)/4 + 1j*0.6*k*rc
        p_end = 1
        U_end = p_end/Z_L
        p_0,U_0 = A*p_end+B*U_end, C*p_end+D*U_end
        Ze = p_0/U_0
    else :
        Ze = B/D
        
    peaks,_ = sig.find_peaks(np.abs(Ze))

    Ym = 1/np.abs(Ze)[peaks]
    fr_m = f[peaks]
    
    Fm = 2*c0/l
    
    return Fm, Ym, fr_m

In [36]:
c0 = 340
# gamme chromatique qui part du do3
frequencies = np.array([130.8, 138.5, 146.8, 155.5, 164.8, 174.6, 184.9, 195.9,207.6,220, 223,246.9, 261])
# frequencies = np.array([261.63, 277.18, 293.66, 311.13, 329.63, 349.23, 369.99, 392.00, 145.3, 440, 466.16, 493.88])
names = ['C2','D2b', 'D2','E2b','E2', 'F2', 'G2b', 'G2', 'A2b', 'A2', 'B2b', 'B2', 'C3']
lengths = c0 / (4 * frequencies)
rc = 0.02
n_modes = 5

file_content = ""
coeffs = []
for idx, l in enumerate(lengths):
    Fm, Ym, fr_m = coeffs_modaux_bois(c0, l, rc)
    Ym = Ym[:n_modes]
    fr_m = fr_m[:n_modes]
    file_content += f"{names[idx]},"
    file_content += f" {Fm:.4f} "
    for Y in Ym:
        file_content += f" {Y:.4f}"
    for freq in fr_m:
        file_content += f" {freq:.2f}"
    file_content += ";\n"

fname=f"coeffs_{n_modes}_modes.txt"
with open(fname, "w") as f:
    f.write(file_content)    